In [1]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini')
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000020F57A27B30>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000020F57DD62D0>, root_client=<openai.OpenAI object at 0x0000020F55F84680>, root_async_client=<openai.AsyncOpenAI object at 0x0000020F578B35C0>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [5]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

class EmailParser(BaseModel):
    sender_name : str = Field(description='이메일 송신자') # 보낸사람
    title : str = Field(description='이메일 제목') # 제목
    content : str = Field(description='이메일 내용') # 내용
    
parser = JsonOutputParser(pydantic_object=EmailParser)
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"sender_name": {"description": "이메일 송신자", "title": "Sender Name", "type": "string"}, "title": {"description": "이메일 제목", "title": "Title", "type": "string"}, "content": {"description": "이메일 내용", "title": "Content", "type": "string"}}, "required": ["sender_name", "title", "content"]}\n```'

In [7]:
prompt_temp = PromptTemplate(
    template='이메일을 읽고 정보를 추출하세요.\n{format_instructions}\n{email}',
    input_variables=['email'],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)
prompt_temp

PromptTemplate(input_variables=['email'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"sender_name": {"description": "이메일 송신자", "title": "Sender Name", "type": "string"}, "title": {"description": "이메일 제목", "title": "Title", "type": "string"}, "content": {"description": "이메일 내용", "title": "Content", "type": "string"}}, "required": ["sender_name", "title", "content"]}\n```'}, template='이메일을 읽고 정보를 추출하세요.\n{format_instructions}\n{email}')

In [8]:
prompt = prompt_temp.format(email='보내는사람: 김철수\n제목: 금일자회의록(9/8)\n회의내용: 오늘 00팀과 진행한 회의록입니다.')
prompt

'이메일을 읽고 정보를 추출하세요.\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"sender_name": {"description": "이메일 송신자", "title": "Sender Name", "type": "string"}, "title": {"description": "이메일 제목", "title": "Title", "type": "string"}, "content": {"description": "이메일 내용", "title": "Content", "type": "string"}}, "required": ["sender_name", "title", "content"]}\n```\n보내는사람: 김철수\n제목: 금일자회의록(9/8)\n회의내용: 오늘 00팀과 진행한 회의록입니다.'

In [9]:
response = llm.invoke(prompt)
response

AIMessage(content='```json\n{\n  "sender_name": "김철수",\n  "title": "금일자회의록(9/8)",\n  "content": "오늘 00팀과 진행한 회의록입니다."\n}\n```', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 265, 'total_tokens': 313, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bda4d3a2c', 'id': 'chatcmpl-CDMidYYUrMWkF1mlFq0nDb7Fgw37j', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--0e7efba4-70d8-4e2f-a347-d08ef6d3be78-0', usage_metadata={'input_tokens': 265, 'output_tokens': 48, 'total_tokens': 313, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})